In [187]:
import pandas as pd

In [188]:
data=pd.read_csv('RTA Dataset.csv')
df=data.copy()

Kiểu dữ liệu của mỗi cột

In [189]:
cols_name = df.columns
def open_object_dtype(s):
    dtypes = set()
    
    #Ta sẽ append lần lượt các type của từng s[i] vào trong set
    ##Thì chắc chắn rằng bên trong dtypes sẽ chỉ có một kiểu duy nhất, cho nên rằng set này chỉ chứa 1 phần tử
    
    ##Chúng ta lấy ra một series dùng apply với hàm type lên tất cả các s[i]
    object_series = s.apply(type)
    
    ##Sau đó dùng chính hàm dtypes.add để apply vô series object_series vừa xử lý xong
    object_series.apply(dtypes.add)
    return dtypes

##Vì trong danh sách các cột, số lượng object khá nhiều cho nên rằng chúng ta sẽ loop qua các danh sách
cols_type_dict = {}
for col_name in cols_name:
    if df[col_name].dtype == 'object':
        cols_type_dict[col_name] = open_object_dtype(df[col_name])
        
cols_type_dict

{'Time': {str},
 'Day_of_week': {str},
 'Age_band_of_driver': {str},
 'Sex_of_driver': {str},
 'Educational_level': {float, str},
 'Vehicle_driver_relation': {float, str},
 'Driving_experience': {float, str},
 'Type_of_vehicle': {float, str},
 'Owner_of_vehicle': {float, str},
 'Service_year_of_vehicle': {float, str},
 'Defect_of_vehicle': {float, str},
 'Area_accident_occured': {float, str},
 'Lanes_or_Medians': {float, str},
 'Road_allignment': {float, str},
 'Types_of_Junction': {float, str},
 'Road_surface_type': {float, str},
 'Road_surface_conditions': {str},
 'Light_conditions': {str},
 'Weather_conditions': {str},
 'Type_of_collision': {float, str},
 'Vehicle_movement': {float, str},
 'Casualty_class': {str},
 'Sex_of_casualty': {str},
 'Age_band_of_casualty': {str},
 'Casualty_severity': {str},
 'Work_of_casuality': {float, str},
 'Fitness_of_casuality': {float, str},
 'Pedestrian_movement': {str},
 'Cause_of_accident': {str},
 'Accident_severity': {str}}

Có một số cột chứa hai kiểu dữ liệu là float và string nên ta cần tiền xử lý kiểu dữ liệu. Một số giá trị nhận kiểu float vì là Nan. Nên ở các cột đó ta cần chuyển về string và xử lý các chỗ bị Nan.

In [190]:
def isNan(string):
    return string!=string

Thể hiện phần trăm bị thiếu dữ liệu của các cột

In [191]:
df.isnull().sum() * 100 / len(df) 


Time                            0.000000
Day_of_week                     0.000000
Age_band_of_driver              0.000000
Sex_of_driver                   0.000000
Educational_level               6.016564
Vehicle_driver_relation         4.701202
Driving_experience              6.731082
Type_of_vehicle                 7.713543
Owner_of_vehicle                3.913608
Service_year_of_vehicle        31.893472
Defect_of_vehicle              35.945112
Area_accident_occured           1.940565
Lanes_or_Medians                3.126015
Road_allignment                 1.152972
Types_of_Junction               7.202014
Road_surface_type               1.396557
Road_surface_conditions         0.000000
Light_conditions                0.000000
Weather_conditions              0.000000
Type_of_collision               1.258525
Number_of_vehicles_involved     0.000000
Number_of_casualties            0.000000
Vehicle_movement                2.500812
Casualty_class                  0.000000
Sex_of_casualty 

Xóa các cột bị thiếu dữ liệu quá nhiều, hoặc các cột chứa thông tin không quá quan trọng về các vụ tai nạn cũng như nguyên nhân.

In [192]:
df = df.drop(columns=['Casualty_class','Sex_of_casualty',
                      'Age_band_of_casualty','Casualty_severity',
                      'Fitness_of_casuality','Defect_of_vehicle',
                      'Pedestrian_movement','Work_of_casuality',
                      'Work_of_casuality','Service_year_of_vehicle',
                      'Vehicle_driver_relation','Owner_of_vehicle'])

In [193]:
#Chuyển cột Time sang kiểu dữ liệu thời gian, cụ thể là hour.
df['Time'] = pd.to_datetime(df['Time'],format= '%H:%M:%S' ).dt.time

In [194]:
#Điền chỗ khuyết bằng mode
df['Educational_level'].fillna(df['Educational_level'].mode()[0],inplace=True)

In [195]:
#Chuyển cột Type_of_vehicle thành kiểu string để xóa bỏ các ký tự sau dấu mở ngoặc
df['Type_of_vehicle'] = df['Type_of_vehicle'].str.replace(r'\([^)]*\)', '',regex=True).str.strip()
df['Type_of_vehicle'].fillna(df['Type_of_vehicle'].mode()[0],inplace=True) 

In [196]:
def transform_age(age_type):
	if age_type == 'Below 1yr':
		age_type = '< 1'
	elif age_type == '1-2yr':
		age_type = (1,2)
	elif age_type == '2-5yr':
		age_type = (2,5)
	elif age_type == '5-10yr':
		age_type = (5,10)
	elif age_type == 'Above 10yr':
		age_type = '> 10'
	return age_type
df['Driving_experience'] = df['Driving_experience'].apply(transform_age)
df['Driving_experience'] = df['Driving_experience'].fillna('unknown')

In [197]:
#Thay thế các giá trị khuyết bằng mode
df['Area_accident_occured'].fillna(df['Area_accident_occured'].mode()[0],inplace=True)
df['Lanes_or_Medians'].fillna(df['Lanes_or_Medians'].mode()[0],inplace=True)
df['Road_allignment'].fillna(df['Road_allignment'].mode()[0],inplace=True)
df['Road_surface_type'].fillna(df['Road_surface_type'].mode()[0],inplace=True)
df['Type_of_collision'].fillna(df['Type_of_collision'].mode()[0],inplace=True)
df['Vehicle_movement'].fillna(df['Vehicle_movement'].mode()[0],inplace=True)
df['Types_of_Junction'].fillna(df['Types_of_Junction'].mode()[0],inplace=True)    

In [198]:
df['Accident_severity'] = df['Accident_severity'].replace(' Injury','',regex=True)

Không còn cột nào bị thiếu.

In [199]:
df.isnull().sum() * 100 / len(df) 

Time                           0.0
Day_of_week                    0.0
Age_band_of_driver             0.0
Sex_of_driver                  0.0
Educational_level              0.0
Driving_experience             0.0
Type_of_vehicle                0.0
Area_accident_occured          0.0
Lanes_or_Medians               0.0
Road_allignment                0.0
Types_of_Junction              0.0
Road_surface_type              0.0
Road_surface_conditions        0.0
Light_conditions               0.0
Weather_conditions             0.0
Type_of_collision              0.0
Number_of_vehicles_involved    0.0
Number_of_casualties           0.0
Vehicle_movement               0.0
Cause_of_accident              0.0
Accident_severity              0.0
dtype: float64

In [200]:
df.to_csv("AfterProcessing.csv")